<a href="https://colab.research.google.com/github/katrinag2004/ds2002assignments/blob/main/financeapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import requests
import pandas as pd

# Your API key
apikey = "uWYBqhtqtN2OZSAnQ6b54Hhuc3sPlOA5niRUY59e"

def fetch_stock_data(symbols):
    url = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols": symbols}
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    stock_data = []
    if 'quoteResponse' in data and 'result' in data['quoteResponse']:
        for stock in data['quoteResponse']['result']:
            stock_data.append({
                'Stock Ticker': stock.get('symbol', 'N/A'),
                'Company': stock.get('longName', 'N/A'),
                'Current Market Price': stock.get('regularMarketPrice', 'N/A')
            })
    return stock_data


def flatten_data(data):
    flat_data = {}
    for key, value in data.items():
        flat_data[key] = value.get('raw', value) if isinstance(value, dict) else value
    return flat_data


def fetch_additional_data(symbol, module):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    querystring = {"modules": module}
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    if 'quoteSummary' in data and 'result' in data['quoteSummary']:
        return flatten_data(data['quoteSummary']['result'][0].get(module, {}))
    else:
        print("Error: Could not retrieve additional data.")
        return {}


def fetch_trending_stocks():
    url = "https://yfapi.net/v1/finance/trending/US"
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers)
    data = response.json()

    trending_stocks = []
    if 'finance' in data and 'result' in data['finance']:
        for stock in data['finance']['result'][0]['quotes']:
            trending_stocks.append({
                'Stock Ticker': stock.get('symbol', 'N/A'),
                'Company': stock.get('shortName', stock.get('longName', 'N/A')),
                'Current Market Price': stock.get('regularMarketPrice', 'N/A'),
                '52 Week High': stock.get('fiftyTwoWeekHigh', 'N/A'),
                '52 Week Low': stock.get('fiftyTwoWeekLow', 'N/A')
            })
    return trending_stocks


if __name__ == "__main__":
    symbols = input("Enter stock symbols separated by commas: ")
    stock_data = fetch_stock_data(symbols)
    if stock_data:
        for stock in stock_data:
            print(f"Stock Ticker: {stock['Stock Ticker']}, Company: {stock['Company']}, Current Market Price: ${stock['Current Market Price']}")

    symbol = input("Enter a stock symbol for additional data: ")
    module = input("Choose a module (summaryDetail, financialData, defaultKeyStatistics): ")
    additional_data = fetch_additional_data(symbol, module)
    if additional_data:
        df = pd.DataFrame([additional_data])
        print(df)

    trending_stocks = fetch_trending_stocks()
    if trending_stocks:
        df = pd.DataFrame(trending_stocks)
        print(df)

# This version handles missing data gracefully and flattens nested fields for easier readability in the DataFrame! 🚀


Enter stock symbols separated by commas: AAPL,GOOG
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.04
Stock Ticker: GOOG, Company: Alphabet Inc., Current Market Price: $173.94
Enter a stock symbol for additional data: AEO
Choose a module (summaryDetail, financialData, defaultKeyStatistics): summaryDetail
   maxAge  priceHint  previousClose   open  dayLow  dayHigh  \
0       1          2          13.36  13.56   13.16   13.645   

   regularMarketPreviousClose  regularMarketOpen  regularMarketDayLow  \
0                       13.36              13.56                13.16   

   regularMarketDayHigh  ...  toCurrency  lastMarket  coinMarketCapLink  \
0                13.645  ...        None        None               None   

   volume24Hr  volumeAllCurrencies  circulatingSupply  algorithm  maxSupply  \
0          {}                   {}                 {}       None         {}   

   startDate  tradeable  
0         {}      False  

[1 rows x 55 columns]
   Stock Ticker